# GDELT Knowledge Graph data pull

## From raw data

In [ ]:
import pandas as pd
import numpy as np
import re
import time

In [26]:
#assign fields name
fields = ['GKGRECORDID','DATE','SOURCECOLLECTIONIDENTIFIER','SOURCECOMMONNAME','DOCUMENTIDENTIFIER',
          'V1COUNTS','V2COUNTS','THEMES','V2THEMES','LOCATIONS','V2LOCATIONS',
          'PERSONS','V2PERSONS','ORGANIZATIONS','V2ORGANIZATIONS','TONE','V2DATES',
          'GCAM','SHARINGIMAGE','RELATEDIMAGES','SOCIALIMAGEEMBEDS','SOCIALVIDEOEMBEDS',
          'QUOTATIONS','ALLNAMES','AMOUNTS','TRANSLATIONINFO','EXTRASXML']

In [ ]:
def extract_us_locations(full_loc_str):
    loc_str_list = full_loc_str.split(';')
    loc_us_list = []
    loc_us_dict = {}
    for loc_str in loc_str_list:
        if int(loc_str.split('#')[0]) == 2 or int(loc_str.split('#')[0]) == 3:
            #loc_us_list.append((loc_str.split('#')[0],loc_str.split('#')[1],loc_str.split('#')[5],loc_str.split('#')[6]))
            loc_us_dict[loc_str.split('#')[1]] = (loc_str.split('#')[0],loc_str.split('#')[1],loc_str.split('#')[5],loc_str.split('#')[6])
    return loc_us_dict

In [ ]:
def extract_theme_by_key(full_theme_str,key):
    theme_str_list = full_theme_str.split(';')
    flag = False
    match_themes = []
    for theme_str in theme_str_list:
        res = re.findall(f'(.*?{key}.*?),',theme_str)
        if len(res) > 0:
            match_themes.append(res[0])
            flag=True
    return flag,match_themes

In [ ]:
#test performance for one file
gkg_pull_one = pd.read_csv('20201217153000.gkg.csv',sep='\t',engine='python',header=None)
gkg_pull_one.columns = fields
results = []
for record_tuple in gkg_pull_one[gkg_pull_one.SOURCECOLLECTIONIDENTIFIER==1].itertuples():
    full_theme_str = record_tuple[9]
    if not full_theme_str or full_theme_str is np.nan:
        #no theme
        pass
    else:
        #print(full_theme_str)
        flag,match_themes = extract_theme_by_key(full_theme_str,'CLIMATE.?CHANGE')
        #flag2,match_themes2 = extract_theme_by_key(full_theme_str,'CLIMATECHANGE')
        
        if flag:
            #print(match_themes)
            full_loc_str = record_tuple[11]
            #print(full_loc_str)
            if not full_loc_str or full_loc_str is np.nan:
                pass
            else:
                us_locations = extract_us_locations(full_loc_str)
                if (len(us_locations) > 0):
                    results.append((record_tuple[1],record_tuple[5],us_locations))

len(results)

In [ ]:
#get all gkg file paths and download latest 500
gkg_file_list = []
with open('masterfilelist.txt') as fo:
    lines = fo.readlines()
for line in lines[-500:]:
    url = line.split(' ')[-1].rstrip('\n')
    res = re.match('.*.gkg..*',url)
    if res:
        #print(url)
        gkg_file_list.append(url)

# extract geoinformation for climate related news
results = []
error_fpaths = []
t0 = time.time()
for fpath in gkg_file_list:
    try:
        gkg_pull_one = pd.read_csv(fpath,sep='\t',engine='python',header=None,encoding='latin_1')
    except:
        print(f'error met {fpath}')
        error_fpaths.append(fpath)
        continue
    gkg_pull_one.columns = fields
    #test performance for one file
    for record_tuple in gkg_pull_one[gkg_pull_one.SOURCECOLLECTIONIDENTIFIER == 1].itertuples():
        full_theme_str = record_tuple[9]
        #print(full_theme_str)
        if not full_theme_str or full_theme_str is np.nan:
            #no theme
            pass
        else:
            #print(len(full_theme_str))
            flag,match_themes = extract_theme_by_key(full_theme_str,'CLIMATE.?CHANGE')
            if flag:
                full_loc_str = record_tuple[11]
                if not full_loc_str or full_loc_str is np.nan:
                    pass
                else:
                    us_locations = extract_us_locations(full_loc_str)
                    if (len(us_locations) > 0):
                        results.append((record_tuple[1],record_tuple[5],us_locations))
    t1 = time.time()
    print(f'{fpath} accumulated run time {t1-t0}s')
len(results)    

## From Google BigQuery

In [ ]:
import os
print(os.getcwd())

In [ ]:
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd
client = bigquery.Client()
bqstorageclient = bigquery_storage.BigQueryReadClient()

# Perform a query.
QUERY = (

'''
    WITH subset_query AS (
  SELECT *  from gdelt-bq.gdeltv2.gkg 
  where DATE>20170302000000 and DATE < 20170304000000 
 limit 200
)
    select gid,regex_extr, ANY_VALUE(code) as level_code,ANY_VALUE(url) as url,ANY_VALUE(location) as raw_loc from 
    (select url,gid,location,regexp_extract(location, r'^[2-3]#(.*?)#') as regex_extr,
    regexp_extract(location, r'^([2-3])#.*?') as code 
    from
    (select subset_query.GKGRECORDID as gid ,subset_query.DocumentIdentifier as url, SPLIT(subset_query.V2Locations, ';') as split_loc
    
    from subset_query) CROSS JOIN UNNEST(split_loc) as location)
    where regex_extr IS NOT NULL
    group by gid,regex_extr
'''

)
#print(QUERY)

In [ ]:
dataframe = (
    client.query(QUERY)
    .result()
    .to_dataframe(bqstorage_client=bqstorageclient)
)
#print(dataframe.head())